## 시퀀스투시퀀스(Sequence-to-Sequence, seq2seq)

시퀀스-투-시퀀스(Sequence-to-Sequence)는 입력된 시퀀스로부터 다른 도메인의 시퀀스를 출력하는 모델

챗봇(Chatbot), 기계 번역(Machine Translation), 내용 요약(Text Summarization), STT(Speech to Text) 등 다양한 분야에서 사용

seq2seq 모델은 입력 -> 인코더(Encoder) -> CONTEXT -> 디코더(Decoder) -> 출력 형태의 구조를 가짐

인코더와 디코더라는 크게 두 개로 구성된 아키텍처로 구성

인코더는 입력 문장의 모든 단어들을 순차적으로 입력받고

모든 단어 정보들을 압축해서 하나의 벡터로 만듬

이를 컨텍스트 벡터(context vector)라 함

입력 문장의 정보가 하나의 컨텍스트 벡터로 모두 압축되면 인코더는 컨텍스트 벡터를 디코더로 전송

디코더는 컨텍스트 벡터를 받아서 한 개씩 순차적으로 출력

## seq2seq의 동작 과정

인코더 아키텍처와 디코더 아키텍처의 내부는 두 개의 RNN 아키텍처

입력 문장을 받는 RNN 셀을 인코더

출력 문장을 출력하는 RNN 셀을 디코더

실제로는 성능 문제로 인해 바닐라 RNN이 아니라 LSTM 셀 또는 GRU 셀들로 구성

입력 문장은 단어 토큰화를 통해서 단어 단위로 쪼개지고 단어 토큰 각각은 RNN 셀의 각 시점의 입력으로 사용

인코더 RNN 셀은 모든 단어를 입력받은 뒤에 인코더 RNN 셀의 마지막 시점의 은닉 상태를 디코더 RNN 셀로 넘겨주며 이를 컨텍스트 벡터라 함

컨텍스트 벡터는 디코더 RNN 셀의 첫번째 은닉 상태로 사용

### 테스트 단계

디코더는 초기 입력으로 문장의 시작을 의미하는 심볼 \<sos\>가 들어감

디코더는 \<sos\>가 입력되면, 다음에 등장할 확률이 높은 단어를 예측

현재 시점의 예측된 결과는 다음 시점의 입력으로 사용하고 이를 반복

즉 디코더는 기본적으로 다음에 올 단어를 예측하고, 그 예측한 단어를 다음 시점의 RNN 셀의 입력으로 넣는 행위를 반복

이 행위는 문장의 끝을 의미하는 심볼인 \<eos\>가 다음 단어로 예측될 때까지 반복됨

### 훈련 단계와 교사 강요

seq2seq는 훈련 과정과 테스트 과정이 작동방식이 다름

훈련 과정에서는 디코더에게 인코더가 보낸 컨텍스트 벡터와 실제 정답 상황인 

\<sos\> 문장을 입력받았을때 문장 \<eos\>가 나와야 된다고 정답을 알려주면서 훈련, 이를 이를 교사 강요(teacher forcing)라 함

반면 테스트 과정에서는 디코더는 오직 컨텍스트 벡터와 \<go\>만을 입력으로 받은 후에 다음에 올 단어를 예측하고, 그 단어를 다음 시점의 RNN 셀의 입력으로 넣는 행위를 반복

### 임베딩 층(Embedding layer)

기계는 텍스트보다 숫자를 잘 처리, 

자연어 처리에서는 텍스트를 벡터로 바꾸는 방법으로 워드 임베딩을 사용

seq2seq에서 사용되는 모든 단어들은 워드 임베딩을 통해 임베딩 벡터로서 표현된 임베딩 벡터

### 디코더

디코더는 인코더의 마지막 RNN 셀의 은닉 상태인 컨텍스트 벡터를 첫번째 은닉 상태의 값으로 사용

디코더의 첫번째 RNN 셀은 이 첫번째 은닉 상태의 값과, 현재 t에서의 입력값인 \<sos\>로부터, 다음에 등장할 단어를 예측

이 예측된 단어는 다음 시점인 t+1 RNN에서의 입력값이 되고

이 t+1에서의 RNN 또한 이 입력값과 t에서의 은닉 상태로부터 t+1에서의 출력 벡터, 다시 다음에 등장할 단어를 예측하게 될 것

seq2seq 모델은 선택될 수 있는 모든 단어들로부터 하나의 단어를 골라서 예측

이를 예측하기 위해서 쓸 수 있는 함수로 softmax 함수를 사용

디코더에서 각 시점(time step)의 RNN 셀에서 출력 벡터가 나오면, 

해당 벡터는 소프트맥스 함수를 통해 출력 시퀀스의 각 단어별 확률값을 반환하고, 디코더는 출력 단어를 결정

## Seq2Seq를 이용한 번역기 구현

seq2seq를 이용한 기계 번역기 구현

실제 서비스되는 번역기는 어텐션 메커니즘을 사용해 구현되며, 최소 수백만 개의 데이터가 필요

### 데이터 로드 및 전처리

http://www.manythings.org/anki

실제 성능이 좋은 기계 번역기를 구현하려면 방대한 데이터가 필요

seq2seq를 간단히 실습해보는 수준의 간단한 기계 번역기를 구현

기계 번역기를 훈련시키기 위해서는 훈련 데이터로 병렬 코퍼스(parallel corpus)가 필요

병렬 코퍼스란, 두 개 이상의 언어가 병렬적으로 구성된 코퍼스를 의미

병렬 데이터라고 하면 앞서 수행한 태깅 작업 챕터의 개체명 인식과 같은 데이터를 생각할 수 있지만 

태깅 작업의 병렬 데이터와 seq2seq가 사용하는 병렬 데이터는 성격이 다름

태깅 작업의 병렬 데이터는 쌍이 되는 데이터와 레이블이 길이가 동일하였으나 여기서는 쌍이 된다고 해서 반드시 길이가 같지는 않음

seq2seq는 기본적으로 입력 시퀀스와 출력 시퀀스의 길이가 다를 수 있다고 가정

예를 들어, seq2seq로 구현할 수 있는 또 다른 예제인 챗봇을 만든다고 가정한다면, 대답의 길이가 질문의 길이와 항상 똑같아야 한다고하면 그 또한 이상

In [99]:
import re
import os
import unicodedata
import urllib3
import zipfile
import shutil
import numpy as np
import pandas as pd
import torch
from collections import Counter
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

In [100]:
# 사용할 총 데이터의 양
num_samples = 150000

In [101]:
def unicode_to_ascii(s):
    # 프랑스어 악센트(accent) 삭제
    # 예시 : 'déjà diné' -> deja dine
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [102]:
def preprocess_sentence(sent):
    # 악센트 삭제 함수 호출
    sent = unicode_to_ascii(sent.lower())

    # 단어와 구두점 사이에 공백을 만듭니다.
    # Ex) "he is a boy." => "he is a boy ."
    sent = re.sub(r"([?.!,¿])", r" \1", sent)

    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환합니다.
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)

    # 다수 개의 공백을 하나의 공백으로 치환
    sent = re.sub(r"\s+", " ", sent)
    return sent

In [103]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []

    with open("fra.txt", "r") as lines:
        for i, line in enumerate(lines):
            # source 데이터와 target 데이터 분리
            src_line, tar_line, _ = line.strip().split('\t')

            # source 데이터 전처리
            src_line = [w for w in preprocess_sentence(src_line).split()]

            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_in = [w for w in ("<sos> " + tar_line).split()]
            tar_line_out = [w for w in (tar_line + " <eos>").split()]

            encoder_input.append(src_line)
            decoder_input.append(tar_line_in)
            decoder_target.append(tar_line_out)

            if i == num_samples - 1:
                break

    return encoder_input, decoder_input, decoder_target

In [104]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장 :', en_sent)
print('전처리 후 영어 문장 :',preprocess_sentence(en_sent))
print('전처리 전 프랑스어 문장 :', fr_sent)
print('전처리 후 프랑스어 문장 :', preprocess_sentence(fr_sent))

전처리 전 영어 문장 : Have you had dinner?
전처리 후 영어 문장 : have you had dinner ?
전처리 전 프랑스어 문장 : Avez-vous déjà diné?
전처리 후 프랑스어 문장 : avez vous deja dine ?


In [105]:
# 전처리 수행
# 훈련 시 사용할 디코더의 입력 시퀀스와 출력 시퀀스를 따로 분리하여 저장
# 입력 시퀀스에는 시작을 의미하는 토큰인 <sos>를 추가
# 출력 시퀀스에는 종료를 의미하는 토큰인 <eos>를 추가
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print('인코더의 입력 :',sents_en_in[:5])
print('디코더의 입력 :',sents_fra_in[:5])
print('디코더의 레이블 :',sents_fra_out[:5])

인코더의 입력 : [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
디코더의 입력 : [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
디코더의 레이블 : [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]


훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 

이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 하는 방법을 사용

그 이유는 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면,

현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 함

이런 상황이 반복되면 훈련 시간이 느려짐

이 상황을 원하지 않는다면 이전 시점의 디코더 셀의 예측값 대신 실제값을 현재 시점의 디코더 셀의 입력으로 사용하는 방법을 사용할 수 있음

이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제값을 입력으로 주는 방법을 교사 강요라고 함

In [106]:
# 단어 집합(Vocabulary) 생성 함수
# 패딩 토큰을 위한 <PAD> 토큰은 0번, OOV에 대응하기 위한 <UNK> 토큰은 1번에 할당
# 빈도수가 가장 높은 단어는 정수가 2번, 빈도수가 두번 째로 많은 단어는 정수 3번이 할당
def build_vocab(sents):
    word_list = []

    for sent in sents:
        for word in sent:
            word_list.append(word)

    # 각 단어별 등장 빈도를 계산하여 등장 빈도가 높은 순서로 정렬
    word_counts = Counter(word_list)
    vocab = sorted(word_counts, key=word_counts.get, reverse=True)

    word_to_index = {}
    word_to_index['<PAD>'] = 0
    word_to_index['<UNK>'] = 1

    # 등장 빈도가 높은 단어일수록 낮은 정수를 부여
    for index, word in enumerate(vocab) :
        word_to_index[word] = index + 2

    return word_to_index

In [107]:
# 구현 방식에 따라서는 하나의 단어 집합으로 만들어도 상관없음
# 영어 단어 집합
src_vocab = build_vocab(sents_en_in)
# 프랑스어 단어 집합
tar_vocab = build_vocab(sents_fra_in + sents_fra_out)

src_vocab_size = len(src_vocab)
tar_vocab_size = len(tar_vocab)
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기 : 10905, 프랑스어 단어 집합의 크기 : 18048


In [108]:
index_to_src = {v: k for k, v in src_vocab.items()}
index_to_tar = {v: k for k, v in tar_vocab.items()}

In [109]:
# 정수로부터 단어를 얻는 딕셔너리 생성
def texts_to_sequences(sents, word_to_index):
    encoded_X_data = []
    for sent in tqdm(sents):
        index_sequences = []
        for word in sent:
            try:
                index_sequences.append(word_to_index[word])
            except KeyError:
                index_sequences.append(word_to_index['<UNK>'])
        encoded_X_data.append(index_sequences)
    return encoded_X_data

In [110]:
encoder_input = texts_to_sequences(sents_en_in, src_vocab)
decoder_input = texts_to_sequences(sents_fra_in, tar_vocab)
decoder_target = texts_to_sequences(sents_fra_out, tar_vocab)

100%|██████████| 150000/150000 [00:00<00:00, 281891.56it/s]


In [111]:
# 상위 샘플에 대해서 정수 인코딩 전, 후 문장 출력
# 인코더 입력이므로 <sos>나 <eos>가 없음
for i, (item1, item2) in zip(range(10), zip(sents_en_in, encoder_input)):
    print(f"Index: {i}, 정수 인코딩 전: {item1}, 정수 인코딩 후: {item2}")

Index: 0, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [45, 2]
Index: 1, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [45, 2]
Index: 2, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [45, 2]
Index: 3, 정수 인코딩 전: ['go', '.'], 정수 인코딩 후: [45, 2]
Index: 4, 정수 인코딩 전: ['hi', '.'], 정수 인코딩 후: [1961, 2]
Index: 5, 정수 인코딩 전: ['hi', '.'], 정수 인코딩 후: [1961, 2]
Index: 6, 정수 인코딩 전: ['run', '!'], 정수 인코딩 후: [336, 76]
Index: 7, 정수 인코딩 전: ['run', '!'], 정수 인코딩 후: [336, 76]
Index: 8, 정수 인코딩 전: ['run', '!'], 정수 인코딩 후: [336, 76]
Index: 9, 정수 인코딩 전: ['run', '!'], 정수 인코딩 후: [336, 76]


In [112]:
# 패딩 함수
def pad_sequences(sentences, max_len=None):
    # 최대 길이 값이 주어지지 않을 경우 데이터 내 최대 길이로 패딩
    if max_len is None:
        max_len = max([len(sentence) for sentence in sentences])

    features = np.zeros((len(sentences), max_len), dtype=int)
    for index, sentence in enumerate(sentences):
        if len(sentence) != 0:
            features[index, :len(sentence)] = np.array(sentence)[:max_len]
    return features

In [113]:
encoder_input = pad_sequences(encoder_input)
decoder_input = pad_sequences(decoder_input)
decoder_target = pad_sequences(decoder_target)

In [114]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)

인코더의 입력의 크기(shape) : (150000, 11)
디코더의 입력의 크기(shape) : (150000, 19)
디코더의 레이블의 크기(shape) : (150000, 19)


In [115]:
# 테스트 데이터를 분리하기 전 데이터를 섞음
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)

랜덤 시퀀스 : [46495  5750 95342 ... 11790 99704 82404]


In [116]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [117]:
print([index_to_src[word] for word in encoder_input[10000]])
print([index_to_tar[word] for word in decoder_input[10000]])
print([index_to_tar[word] for word in decoder_target[10000]])

['the', 'chili', 'burnt', 'my', 'tongue', '.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['<sos>', 'le', 'piment', 'm', 'a', 'brule', 'la', 'langue', '.', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['le', 'piment', 'm', 'a', 'brule', 'la', 'langue', '.', '<eos>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [118]:
n_of_val = int(33000*0.1)
print('검증 데이터의 개수 :',n_of_val)

검증 데이터의 개수 : 3300


In [119]:
# 데이터 분리
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [120]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (146700, 11)
훈련 target 데이터의 크기 : (146700, 19)
훈련 target 레이블의 크기 : (146700, 19)
테스트 source 데이터의 크기 : (3300, 11)
테스트 target 데이터의 크기 : (3300, 19)
테스트 target 레이블의 크기 : (3300, 19)


### 번역기 모델링

In [121]:
import torch
import torch.nn as nn
import torch.optim as optim

In [122]:
embedding_dim = 256
hidden_units = 256

In [123]:
# Encoder 클래스는 입력 시퀀스를 받아 해당 시퀀스의 정보를 압축하여 context vector로 변환하는 역할
# Encoder는 임베딩 레이어와 LSTM 레이어로 구성
# 임베딩 레이어는 입력 시퀀스의 각 토큰을 고정 크기의 벡터로 변환
# LSTM 레이어는 시퀀스의 순서 정보를 고려하여 해당 시퀀스를 요약
# Encoder의 forward 메서드는 입력 시퀀스를 받아 LSTM의 hidden state와 cell state를 반환
class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embedding_dim, hidden_units):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(src_vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_units, batch_first=True)

    def forward(self, x):
        # x.shape == (batch_size, seq_len, embedding_dim)
        x = self.embedding(x)
        # hidden.shape == (1, batch_size, hidden_units), cell.shape == (1, batch_size, hidden_units)
        _, (hidden, cell) = self.lstm(x)
        # 인코더의 출력은 hidden state, cell state
        return hidden, cell

In [124]:
# Decoder 클래스는 Encoder에서 생성된 context vector(인코더의 마지막 은닉 상태)를 기반으로 출력 시퀀스를 생성하는 역할
# Decoder는 임베딩 레이어와 LSTM 레이어로 구성
# Decoder의 LSTM은 Encoder에서 전달받은 hidden state와 cell state를 초기 상태로 사용하여 출력 시퀀스를 생성
# 생성된 출력 시퀀스는 fully connected 레이어를 통과하여 각 시점의 출력 토큰에 대한 확률 분포를 구함
# Decoder의 forward 메서드는 입력 시퀀스, hidden state, cell state를 받아 출력 시퀀스, 업데이트된 hidden state와 cell state를 반환
class Decoder(nn.Module):
    def __init__(self, tar_vocab_size, embedding_dim, hidden_units):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(tar_vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_units, batch_first=True)
        self.fc = nn.Linear(hidden_units, tar_vocab_size)

    def forward(self, x, hidden, cell):

        # x.shape == (batch_size, seq_len, embedding_dim)
        x = self.embedding(x)

        # 디코더의 LSTM으로 인코더의 hidden state, cell state를 전달.
        # output.shape == (batch_size, seq_len, hidden_units)
        # hidden.shape == (1, batch_size, hidden_units)
        # cell.shape == (1, batch_size, hidden_units)
        output, (hidden, cell) = self.lstm(x, (hidden, cell))

        # output.shape: (batch_size, seq_len, tar_vocab_size)
        output = self.fc(output)

        # 디코더의 출력은 예측값, hidden state, cell state
        return output, hidden, cell

In [125]:
# Seq2Seq 클래스는 Encoder와 Decoder를 결합하여 전체 모델을 구성
# forward 메서드는 입력 시퀀스(src)와 출력 시퀀스(trg)를 받아 
# Encoder에서 생성된 은닉 상태(hidden state)와 셀 상태(cell state)를 Decoder로 전달하고, 
# Decoder에서 생성된 출력 시퀀스를 반환
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg):
        hidden, cell = self.encoder(src)

        # 훈련 중에는 디코더의 출력 중 오직 output만 사용한다.
        output, _, _ = self.decoder(trg, hidden, cell)
        return output

In [126]:
encoder = Encoder(src_vocab_size, embedding_dim, hidden_units)
decoder = Decoder(tar_vocab_size, embedding_dim, hidden_units)
model = Seq2Seq(encoder, decoder)

In [127]:
# Seq2Seq의 디코더는 기본적으로 각 시점마다 다중 클래스 분류 문제를 푸는것과 같음
# 매 시점마다 프랑스어 단어 집합의 크기(tar_vocab_size)의 선택지에서 단어를 1개 선택하여 이를 이번 시점에서 예측한 단어로 택함
# 다중 클래스 분류 문제이므로 모델 학습을 위해 CrossEntropyLoss 함수를 사용하여 손실을 계산
# Adam 옵티마이저를 사용하여 모델의 파라미터를 최적화
# CrossEntropyLoss의 ignore_index 파라미터는 패딩 토큰에 해당하는 인덱스를 무시하도록 설정
loss_function = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters())

In [128]:
print(model)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10905, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(18048, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=18048, bias=True)
  )
)


In [129]:
# 평가 함수의 입력으로는 평가할 모델(model), 데이터로더(dataloader), 손실 함수(loss_function), 그리고 모델을 실행할 디바이스(device)가 주어짐
def evaluation(model, dataloader, loss_function, device):
    # 모델을 평가 모드로 설정
    # 드롭아웃(dropout)이나 배치 정규화(batch normalization)와 같은 층의 동작을 조정
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_count = 0

    # 그래디언트 계산을 비활성화하여 메모리 사용량을 줄이고 평가 속도를 향상
    with torch.no_grad():
        # 각 배치(batch)에 대해, 인코더 입력(encoder_inputs), 디코더 입력(decoder_inputs), 그리고 디코더 타겟(decoder_targets)을 디바이스로 이동
        for encoder_inputs, decoder_inputs, decoder_targets in dataloader:
            encoder_inputs = encoder_inputs.to(device)
            decoder_inputs = decoder_inputs.to(device)
            decoder_targets = decoder_targets.to(device)

            # 순방향 전파
            # outputs.shape == (batch_size, seq_len, tar_vocab_size)
            outputs = model(encoder_inputs, decoder_inputs)

            # 손실 계산
            # outputs.view(-1, outputs.size(-1))의 shape는 (batch_size * seq_len, tar_vocab_size)
            # decoder_targets.view(-1)의 shape는 (batch_size * seq_len)
            # 출력과 타겟의 차원을 조정하기 위해 view() 함수를 사용
            loss = loss_function(outputs.view(-1, outputs.size(-1)), decoder_targets.view(-1))
            total_loss += loss.item()

            # 정확도 계산 (패딩 토큰 제외)
            # 패딩 토큰(padding token)을 제외한 실제 토큰들에 대해서만 고려
            # 디코더 타겟이 0이 아닌 위치에 대한 마스크(mask)를 생성
            # 출력의 argmax를 취하여 예측된 토큰을 얻고, 이를 디코더 타겟과 비교하여 정확한 예측 수를 계산합니다. 정확한 예측 수와 전체 토큰 수를 누적
            mask = decoder_targets != 0
            total_correct += ((outputs.argmax(dim=-1) == decoder_targets) * mask).sum().item()
            total_count += mask.sum().item()

    # 평균 손실(average loss)과 정확도(accuracy)를 계산하여 반환
    # 평균 손실은 총 손실을 데이터로더의 배치 수로 나누어 계산하고, 정확도는 총 정확도를 총 토큰 수로 나누어 계산
    return total_loss / len(dataloader), total_correct / total_count

In [130]:
encoder_input_train_tensor = torch.tensor(encoder_input_train, dtype=torch.long)
decoder_input_train_tensor = torch.tensor(decoder_input_train, dtype=torch.long)
decoder_target_train_tensor = torch.tensor(decoder_target_train, dtype=torch.long)

encoder_input_test_tensor = torch.tensor(encoder_input_test, dtype=torch.long)
decoder_input_test_tensor = torch.tensor(decoder_input_test, dtype=torch.long)
decoder_target_test_tensor = torch.tensor(decoder_target_test, dtype=torch.long)

In [131]:
# 데이터셋 및 데이터로더 생성
batch_size = 128

train_dataset = TensorDataset(encoder_input_train_tensor, decoder_input_train_tensor, decoder_target_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = TensorDataset(encoder_input_test_tensor, decoder_input_test_tensor, decoder_target_test_tensor)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [132]:
# 학습 설정
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10905, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(18048, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=18048, bias=True)
  )
)

In [133]:
# Training loop
best_val_loss = float('inf')

# 모델 훈련
for epoch in range(num_epochs):
    # 훈련 모드
    model.train()

    for encoder_inputs, decoder_inputs, decoder_targets in train_dataloader:
        encoder_inputs = encoder_inputs.to(device)
        decoder_inputs = decoder_inputs.to(device)
        decoder_targets = decoder_targets.to(device)

        # 기울기 초기화
        optimizer.zero_grad()

        # 순방향 전파
        # outputs.shape == (batch_size, seq_len, tar_vocab_size)
        outputs = model(encoder_inputs, decoder_inputs)

        # 손실 계산 및 역방향 전파
        # outputs.view(-1, outputs.size(-1))의 shape는 (batch_size * seq_len, tar_vocab_size)
        # decoder_targets.view(-1)의 shape는 (batch_size * seq_len)
        loss = loss_function(outputs.view(-1, outputs.size(-1)), decoder_targets.view(-1))
        loss.backward()

        # 가중치 업데이트
        optimizer.step()

    train_loss, train_acc = evaluation(model, train_dataloader, loss_function, device)
    valid_loss, valid_acc = evaluation(model, valid_dataloader, loss_function, device)

    print(f'Epoch: {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Valid Loss: {valid_loss:.4f} | Valid Acc: {valid_acc:.4f}')

    # 검증 손실이 최소일 때 체크포인트 저장
    if valid_loss < best_val_loss:
        print(f'Validation loss improved from {best_val_loss:.4f} to {valid_loss:.4f}. 체크포인트를 저장합니다.')
        best_val_loss = valid_loss
        torch.save(model.state_dict(), 'best_model_checkpoint.pth')

Epoch: 1/20 | Train Loss: 2.2179 | Train Acc: 0.5860 | Valid Loss: 2.3411 | Valid Acc: 0.5768
Validation loss improved from inf to 2.3411. 체크포인트를 저장합니다.
Epoch: 2/20 | Train Loss: 1.5562 | Train Acc: 0.6759 | Valid Loss: 1.7975 | Valid Acc: 0.6501
Validation loss improved from 2.3411 to 1.7975. 체크포인트를 저장합니다.
Epoch: 3/20 | Train Loss: 1.1710 | Train Acc: 0.7382 | Valid Loss: 1.5229 | Valid Acc: 0.6934
Validation loss improved from 1.7975 to 1.5229. 체크포인트를 저장합니다.
Epoch: 4/20 | Train Loss: 0.9152 | Train Acc: 0.7812 | Valid Loss: 1.3697 | Valid Acc: 0.7195
Validation loss improved from 1.5229 to 1.3697. 체크포인트를 저장합니다.
Epoch: 5/20 | Train Loss: 0.7371 | Train Acc: 0.8160 | Valid Loss: 1.2709 | Valid Acc: 0.7359
Validation loss improved from 1.3697 to 1.2709. 체크포인트를 저장합니다.
Epoch: 6/20 | Train Loss: 0.6129 | Train Acc: 0.8421 | Valid Loss: 1.2193 | Valid Acc: 0.7461
Validation loss improved from 1.2709 to 1.2193. 체크포인트를 저장합니다.
Epoch: 7/20 | Train Loss: 0.5238 | Train Acc: 0.8617 | Valid Loss: 

In [134]:
# 모델 로드
model.load_state_dict(torch.load('best_model_checkpoint.pth'))
model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10905, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(18048, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=18048, bias=True)
  )
)

In [135]:
# 검증 데이터에 대한 정확도와 손실 계산
val_loss, val_accuracy = evaluation(model, valid_dataloader, loss_function, device)

print(f'Best model validation loss: {val_loss:.4f}')
print(f'Best model validation accuracy: {val_accuracy:.4f}')

Best model validation loss: 1.1857
Best model validation accuracy: 0.7579


In [136]:
print(tar_vocab['<sos>'])
print(tar_vocab['<eos>'])

3
4


### seq2seq 기계 번역기 동작

seq2seq는 훈련 과정(교사 강요)과 테스트 과정에서의 동작 방식이 다름

테스트 과정을 위해 모델을 다시 설계해주어야 함, 특히 디코더를 수정해야 함

전체적인 번역 단계는 다음과 같음
  1. 번역하고자 하는 입력 문장이 인코더로 입력되어 인코더의 마지막 시점의 은닉 상태와 셀 상태를 얻음
  2. 인코더의 은닉 상태와 셀 상태, 그리고 토큰 \<sos\>를 디코더로 보냄
  3. 디코더가 토큰 \<eos\>가 나올 때까지 다음 단어를 예측하는 행동을 반복

In [137]:
index_to_src = {v: k for k, v in src_vocab.items()}
index_to_tar = {v: k for k, v in tar_vocab.items()}

In [138]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0):
            sentence = sentence + index_to_src[encoded_word] + ' '
    return sentence

In [139]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0 and encoded_word != tar_vocab['<sos>'] and encoded_word != tar_vocab['<eos>']):
            sentence = sentence + index_to_tar[encoded_word] + ' '
    return sentence

In [140]:
print(encoder_input_test[25])
print(decoder_input_test[25])
print(decoder_target_test[25])

[36 23 67  6 45  2  0  0  0  0  0]
[  3  61  55  71  17 382  11 178   2   0   0   0   0   0   0   0   0   0
   0]
[ 61  55  71  17 382  11 178   2   4   0   0   0   0   0   0   0   0   0
   0]


In [145]:
# 테스트 단계에서는 디코더를 매 시점 별로 컨트롤
# 각 시점을 for문을 통해서 컨트롤
# 현재 시점의 예측은 다음 시점의 입력으로 사용, 여기서 사용될 변수는 decoder_input
def decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, max_output_len, int_to_src_token, int_to_tar_token):
    encoder_inputs = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)

    # 인코더의 초기 상태 설정
    hidden, cell = model.encoder(encoder_inputs)

    # 시작 토큰 <sos>을 디코더의 첫 입력으로 설정
    # unsqueeze(0)는 배치 차원을 추가하기 위함.
    decoder_input = torch.tensor([3], dtype=torch.long).unsqueeze(0).to(device)

    decoded_tokens = []

    # for문을 도는 것 == 디코더의 각 시점
    for _ in range(max_output_len):
        output, hidden, cell = model.decoder(decoder_input, hidden, cell)

        # 소프트맥스 회귀를 수행. 예측 단어의 인덱스
        output_token = output.argmax(dim=-1).item()

        # 종료 토큰 <eos>
        if output_token == 4:
            break

        # 각 시점의 단어(정수)는 decoded_tokens에 누적하였다가 최종 번역 시퀀스로 리턴합니다.
        decoded_tokens.append(output_token)

        # 현재 시점의 예측. 다음 시점의 입력으로 사용된다.
        decoder_input = torch.tensor([output_token], dtype=torch.long).unsqueeze(0).to(device)

    return ' '.join(int_to_tar_token[token] for token in decoded_tokens)

In [147]:
for seq_index in [3, 50, 100, 300, 1000, 1500, 3000]:
    input_seq = encoder_input_train[seq_index]
    translated_text = decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, 20, index_to_src, index_to_tar)

    print("입력문장 :",seq_to_src(encoder_input_train[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_train[seq_index]))
    print("번역문장 :",translated_text)
    print("-"*50)

입력문장 : he is a psychic . 
정답문장 : c est un voyant . 
번역문장 : c est un voyant .
--------------------------------------------------
입력문장 : tom is very hard on himself . 
정답문장 : tom est tres dur envers lui meme . 
번역문장 : tom est tres dur envers lui .
--------------------------------------------------
입력문장 : you were removed from the list . 
정답문장 : tu as ete retire de la liste . 
번역문장 : tu as ete retiree de la liste .
--------------------------------------------------
입력문장 : all of us plan to do that . 
정답문장 : nous tous prevoyons de le faire . 
번역문장 : nous allons tous faire de notre mieux .
--------------------------------------------------
입력문장 : the water was turned on . 
정답문장 : le robinet etait ouvert . 
번역문장 : l eau etait glaciale .
--------------------------------------------------
입력문장 : he let me go . 
정답문장 : il me laissa m en aller . 
번역문장 : il m a laisse partir .
--------------------------------------------------
입력문장 : i m not a child . 
정답문장 : je ne suis pas une enfant . 
번역문장 : j

In [148]:
for seq_index in [3, 50, 100, 300, 1001, 1500, 3000]:
    input_seq = encoder_input_test[seq_index]
    translated_text = decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, 20, index_to_src, index_to_tar)

    print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
    print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
    print("번역문장 :",translated_text)
    print("-"*50)

입력문장 : she took him to the store . 
정답문장 : elle l amena au magasin . 
번역문장 : elle l a emmene au magasin .
--------------------------------------------------
입력문장 : what s that mean ? 
정답문장 : qu est ce que ca veut dire ? 
번역문장 : qu est ce que ca veut dire ?
--------------------------------------------------
입력문장 : this is tom . 
정답문장 : c est tom . 
번역문장 : c est tom .
--------------------------------------------------
입력문장 : you have my sympathies . 
정답문장 : tu as ma sympathie . 
번역문장 : vous avez mon choix .
--------------------------------------------------
입력문장 : i know tom was distracted . 
정답문장 : je sais que tom etait distrait . 
번역문장 : je sais que tom etait distrait .
--------------------------------------------------
입력문장 : let s give it a whirl . 
정답문장 : essayons ca . 
번역문장 : essayons !
--------------------------------------------------
입력문장 : we don t need that . 
정답문장 : nous n avons pas besoin de ca . 
번역문장 : nous n avons pas besoin de ca .
---------------------------------------